## Curating Data From Kaggle to Colab

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!unzip chest-xray-pneumonia.zip

## Importing Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, Conv2D, MaxPool2D,GlobalAveragePooling2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image

## Loading Data

In [ ]:
train_datagen = image.ImageDataGenerator(
    rescale = 1/255,
    horizontal_flip = True,
    zoom_range = 0.2,
    shear_range = 0.2
)

train_data = train_datagen.flow_from_directory(directory="/content/chest_xray/chest_xray/train",
                                               target_size = (256, 256),
                                               batch_size = 16,
                                               class_mode = "binary")

In [ ]:
train_data.class_indices

In [ ]:
test_datagen = image.ImageDataGenerator(
    rescale = 1/255
)

test_data = test_datagen.flow_from_directory(directory="/content/chest_xray/chest_xray/test",
                                             target_size = (256, 256),
                                             batch_size = 16,
                                               class_mode = "binary")

In [ ]:
test_data.class_indices

## CNN MODEL

In [ ]:
model = Sequential()

model.add(Conv2D(filters= 32, kernel_size=(3,3),activation="relu",input_shape=(256,256,3)))

model.add(Conv2D(filters= 64, kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters= 128, kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters= 256, kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dropout(rate=0.50))
model.add(Dense(units=1, activation = "sigmoid"))


model.compile(loss=keras.losses.binary_focal_crossentropy, optimizer = "adam", metrics = ["acc"])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_data, steps_per_epoch= 10, epochs = 15, validation_steps =2,
                    validation_data = test_data)

## Prediction on Sample Images

In [ ]:
!pip install utils

In [ ]:
import tensorflow as tf

def load_image(path):
    """Loads an image from the specified path."""
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize([img], (256, 256))
    img = img / 255.0
    return img


img = load_image("/content/chest_xray/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg")
print(img.shape)
predictions = model.predict(img)
predicted_class = np.argmax(predictions[0])
print(predicted_class)


## Prediction On NORMAL

In [ ]:
def read_images_from_folder(folder):
    """Reads all the images from the specified folder."""
    for img_path in os.listdir(folder):
        img = load_image(os.path.join(folder, img_path))
        yield img


if __name__ == "__main__":
    folder = "/content/chest_xray/val/NORMAL"
    for img in read_images_from_folder(folder):
        print(img.shape)
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions[0])
        print(predicted_class)
